# Ethereum's Proposer-Builder Separation: Promises and Realities 

This script was run with the following Python 3.10.12 and the following libary versions: 
- panads 1.5.3
- numpy 1.25.2
- matplotlib 3.7.2
- seaborn 0.12.0

In [ ]:
import pandas as pd
import numpy as np
from  enum import Enum
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
import matplotlib
import seaborn as sns
import pickle


In [ ]:
class Color(Enum):
    bloXrouteR = "#06AED5" 
    bloXrouteM = "#DA3E52" 
    LIDO = "#F8BE57"
    builder0x69 ="#7E6B8F" 
    REST = "#02C39A" 
    Flashbots = "#05668D"
    beaverbuild ='#B118C8'
    Eden = '#FF784F'
    bloXrouteE = '#B98389'
    ethbuilder = '#F2BAC9'
    MEVBuilder1 ='#C1CC99'
    MEVBuilder2 ='#3c096c'

In [ ]:
#plot parameters
plt.rcParams["figure.figsize"] = (3.465,1.8) 
font = {'family' : 'serif', 'size'   : 6.3}
matplotlib.rc('font', **font)   

In [ ]:
#helper class to move y axis factor, obtained from https://stackoverflow.com/questions/45760763/how-to-move-the-y-axis-scale-factor-to-the-position-next-to-the-y-axis-label
class Labeloffset():
    def __init__(self,  ax, label="", axis="y"):
        self.axis = {"y":ax.yaxis, "x":ax.xaxis}[axis]
        self.label=label
        ax.callbacks.connect(axis+'lim_changed', self.update)
        ax.figure.canvas.draw()
        self.update(None)

    def update(self, lim):
        fmt = self.axis.get_major_formatter()
        self.axis.offsetText.set_visible(False)
        self.axis.set_label_text(self.label + " "+ fmt.get_offset() )    

Load Data

In [ ]:

#PBS data
data=pd.read_pickle('data.pkl')
PBSblocks =data[data['PBSBlock']==1]
noPBSblocks =data[data['PBSBlock']==0]

with open('relaysBlockDict.pickle', 'rb') as f:
    relaysBlockDict =pickle.load(f)
with open('relaysDict.pickle', 'rb') as f:
    relaysDict =pickle.load(f)    

#private TXs data   
privateNonPBS=pd.read_pickle('privateNonPBS.pkl')


## PBS Landscape

### Share of Daily User Payments

In [ ]:
proportionOfValue=data[['burnedFee','proposerFee','coinbaseMev','day']]
proportionOfValue=proportionOfValue.groupby(['day']).sum()
proportionOfValue['sum']=proportionOfValue.sum(axis=1)

proportionOfValue.iloc[:,:-1]=proportionOfValue.iloc[:,:-1].div(proportionOfValue.iloc[:,-1], axis=0)

plt.stackplot(proportionOfValue.index.tolist(), proportionOfValue.burnedFee.tolist(), proportionOfValue.proposerFee.tolist(), proportionOfValue.coinbaseMev.tolist(),
              labels=[ "base fee", "priority fee", "direct transfers" ], 
              colors =[Color.bloXrouteM.value, Color.Flashbots.value, Color.MEVBuilder1.value])

ax=plt.gca()
locator = mdates.AutoDateLocator()
formatter = mdates.ConciseDateFormatter(locator)
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(formatter)   
ax.xaxis.set_minor_formatter('')

plt.xlabel('')
plt.ylabel('share of user payments')

plt.ylim(0,1)
handles, labels = ax.get_legend_handles_labels()
lgd = ax.legend(handles,labels,facecolor ="white", framealpha=1, bbox_to_anchor=(0., 1.02, 1., .102), loc=3,  ncol=6, mode="expand", borderaxespad=0.)
ax.set_xlim([pd.to_datetime('2022-09-15'), pd.to_datetime('2023-03-31')])
plt.show()

### Share of PBS Blocks

In [ ]:
dataPBSDaily = data.resample('D').agg({'PBSBlock': ['sum', 'size']}) 
dataPBSDaily['PBSBlock','sum1']  =dataPBSDaily['PBSBlock','size']-dataPBSDaily['PBSBlock','sum']
dataPBSDaily['PBS'] = dataPBSDaily['PBSBlock','sum']/dataPBSDaily['PBSBlock','size']
dataPBSDaily['noPBS'] = dataPBSDaily['PBSBlock','sum1']/dataPBSDaily['PBSBlock','size']

In [ ]:
plt.stackplot(dataPBSDaily.index.tolist(), dataPBSDaily.PBS.tolist(), dataPBSDaily.noPBS.tolist(),
              labels=[ "PBS", "non-PBS", ], 
              colors =[Color.Flashbots.value, Color.bloXrouteM.value])
ax =plt.gca()
locator = mdates.AutoDateLocator()
formatter = mdates.ConciseDateFormatter(locator)
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(formatter)     
plt.xlabel('')
plt.ylabel('share of blocks')

plt.ylim(0,1)
handles, labels = ax.get_legend_handles_labels()
lgd = ax.legend(handles,labels,facecolor ="white", framealpha=1, bbox_to_anchor=(0., 1.02, 1., .102), loc=3,  ncol=6, mode="expand", borderaxespad=0.)
ax.set_xlim([pd.to_datetime('2022-09-15'), pd.to_datetime('2023-03-31')])

plt.show()

In [ ]:
print("Number of PBS blocks: "+str(len(data[data['PBSBlock']==1])))
print("Number of PBS blocks claimed by relays: "+str(len(data[data['builderPubkey']==data['builderPubkey']])))
print("Number of PBS blocks with final payment: "+str(len(data[data['payment']==1])))
print("Number of PBS blocks with final payment: "+str(len(data[data['payment']==1])))
print("Number of PBS blocks without final payment and same builder/proposer: "+str(len(data[(data['payment']==0)&(data['builderFeeRecipient']==data['proposerFeeRecipient'])])))

### Relay Distribution

In [ ]:
relayDictData =pd.DataFrame.from_dict(relaysBlockDict, orient='index').T.unstack().dropna().reset_index(level=1,drop=True)
relayDictData =relayDictData.to_frame()
relayDictData = relayDictData.reset_index()
relayDictData.columns =['block', 'Relay']
relayDictData =relayDictData.groupby(['block',  'Relay']).size().unstack(fill_value=0)
relayDictData =relayDictData.div(relayDictData.sum(axis=1), axis=0)
data['time'] = data.index
dataRelay=relayDictData.merge(data[data['PBSBlock']==1][['block','time']], left_on='block', right_on='block',how ='outer')

dataRelay =dataRelay.set_index('time')
dataRelay.index = pd.to_datetime(dataRelay.index)
dataRelay=dataRelay.drop(['block'],axis=1)
dataRelay['left'] = np.where(dataRelay.sum(axis=1)==0,1, 0)

dataRelay=dataRelay.resample('D').sum()
s = dataRelay.sum()
dataRelayTop=dataRelay[s.sort_values(ascending=False).index[:8]]
dataRelayRest=dataRelay[s.sort_values(ascending=False).index[8:]]
dataRelayRest['rest']=dataRelayRest.sum(axis=1).copy()
dataRelayTop=dataRelayTop.merge(dataRelayRest['rest'], left_on='time', right_on='time',how ='inner')
dataRelayTop=dataRelayTop.merge(blocksPerBuilder['sum'], left_index=True, right_index=True,how ='inner')

dataRelayTop.iloc[:,:-1]=dataRelayTop.iloc[:,:-1].div(dataRelayTop.iloc[:,-1], axis=0)
dataRelayTop.index =pd.to_datetime(dataRelayTop.index)
dataRelayTop =dataRelayTop.rename(columns={
                  "bloXroute (MaxProfit)": "bloXroute (M)"
                  ,"bloXroute (Regulated)": "bloXroute (R)"
                  ,"bloXroute (Ethical)": "bloXroute (E)"
                  })

In [ ]:
ax = dataRelayTop.drop('sum', axis=1).plot.area(linewidth=0,color={'Flashbots': Color.Flashbots.value, 
                                    'bloXroute (M)': Color.bloXrouteM.value, 
                                    'Blocknative': Color.beaverbuild.value, 
                                    'bloXroute (R)': Color.bloXrouteR.value, 
                                    'Eden': Color.Eden.value, 
                                    'bloXroute (E)': Color.bloXrouteE.value, 
                                    'UltraSound': Color.MEVBuilder2.value, 
                                    'GnosisDAO': Color.MEVBuilder1.value, 
                                    'rest': Color.REST.value})
locator = mdates.AutoDateLocator()
formatter = mdates.ConciseDateFormatter(locator)
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(formatter)   
ax.xaxis.set_minor_formatter('')
ax.tick_params(which='minor', length=0, color='r')
plt.xlabel('')
plt.ylabel('share of blocks')
plt.ylim(0,1)
handles, labels = ax.get_legend_handles_labels()
lgd = ax.legend(handles,labels,facecolor ="white", framealpha=1, bbox_to_anchor=(0., 1.02, 1., .102), loc=3,  ncol=3, mode="expand", borderaxespad=0.)
ax.set_xlim([pd.to_datetime('2022-09-15'), pd.to_datetime('2023-03-31')])
plt.show()

### Builder Distribution

In [ ]:
data =data.replace("\\xa1dead01e65f0a0eee7b5170223f20c8f0cbf122eac3324d61afbdb33a8885ff8cab2ef514ac2c7698ae0d6289ef27fc",  "Flashbots")
data =data.replace('\\x81babeec8c9f2bb9c329fd8a3b176032fe0ab5f3b92a3f44d4575a231c7bd9c31d10b6328ef68ed1e8c02a3dbc8e80f9',  "Flashbots")
data =data.replace('\\x81beef03aafd3dd33ffd7deb337407142c80fea2690e5b3190cfc01bde5753f28982a7857c96172a75a234cb7bcb994f',  "Flashbots")
data =data.replace('\\xb086acdd8da6a11c973b4b26d8c955addbae4506c78defbeb5d4e00c1266b802ff86ec7457c4c3c7c573fa1e64f7e9e0',  "bloXroute (E)")
data =data.replace('\\x95701d3f0c49d7501b7494a7a4a08ce66aa9cc1f139dbd3eec409b9893ea213e01681e6b76f031122c6663b7d72a331b',  "bloXroute (E)")
data =data.replace('\\x82801ab0556f7df1fb9bb3a61ca84beea8285a8dc3c455a7ea16a8b2993fe06058e0e7d275b28ea5d9f2ae995aa72605',  "bloXroute (E)")
data =data.replace('\\x94aa4ee318f39b56547a253700917982f4b737a49fc3f99ce08fa715e488e673d88a60f7d2cf9145a05127f17dcb7c67',  "bloXroute (M)")
data =data.replace('\\x976e63c505050e25b70b39238990c78ddf0948685eb8c5687d17ba5089541f37dd3c45999f2db449eac298b1d4856013',  "bloXroute (M)")
data =data.replace('\\x8b8edce58fafe098763e4fabdeb318d347f9238845f22c507e813186ea7d44adecd3028f9288048f9ad3bc7c7c735fba',  "bloXroute (M)")
data =data.replace('\\xaa1488eae4b06a1fff840a2b6db167afc520758dc2c8af0dfb57037954df3431b747e2f900fe8805f05d635e9a29717b',  "bloXroute (M)")
data =data.replace('\\x80c7311597316f871363f8395b6a8d056071d90d8eb27defd14759e8522786061b13728623452740ba05055f5ba9d3d5',  "bloXroute (R)")
data =data.replace('\\xb9b50821ec5f01bb19ec75e0f22264fa9369436544b65c7cf653109dd26ef1f65c4fcaf1b1bcd2a7278afc34455d3da6',  "bloXroute (R)")
data =data.replace('\\x965a05a1ba338f4bbbb97407d70659f4cea2146d83ac5da6c2f3de824713c927dcba706f35322d65764912e7756103e2',  "bloXroute (R)")
data =data.replace('\\xa1daf0ab37a9a204bc5925717f78a795fa2812f8fba8bda10b1b27c554bd7dedd46775106facd72be748eea336f514e9',  "Builder 1")
data =data.replace('\\x89783236c449f037b4ad7bae18cea35014187ec06e2daa016128e736739debeafc5fe8662a0613bc4ca528af5be83b3c',  "Builder 1")
data =data.replace('\\xb194b2b8ec91a71c18f8483825234679299d146495a08db3bf3fb955e1d85a5fca77e88de93a74f4e32320fc922d3027',  "builder0x69")
data =data.replace('\\xa971c4ee4ac5d47e0fb9e16be05981bfe51458f14c06b7a020304099c23d2d9952d4254cc50f291c385d15e7cae0cf9d',  "builder0x69")
data =data.replace('\\xa4fb63c2ceeee73d1f1711fadf1c5357ac98cecb999d053be613f469a48f7416999a4da35dd60a7824478661399e6772',  "builder0x69")
data =data.replace('\\xb8fceec09779ff758918a849bfe8ab43cea79f6a98320af0af5b030f6a7850fcc5883cb965d02efb10eed1ffa987e899',  "builder0x69")
data =data.replace('\\x8bc8d110f8b5207e7edc407e8fa033937ddfe8d2c6f18c12a6171400eb6e04d49238ba2b0a95e633d15558e6a706fbe4',  "builder0x69")
data =data.replace('\\xb5d883565500910f3f10f0a2e3a031139d972117a3b67da191ff93ba00ba26502d9b65385b5bca5e7c587273e40f2319',  "beaverbuild")
data =data.replace('\\x96a59d355b1f65e270b29981dd113625732539e955a1beeecbc471dd0196c4804574ff871d47ed34ff6d921061e9fc27',  "beaverbuild")
data =data.replace('\\x8dde59a0d40b9a77b901fc40bee1116acf643b2b60656ace951a5073fe317f57a086acf1eac7502ea32edcca1a900521',  "beaverbuild")
data =data.replace('\\xaec4ec48c2ec03c418c599622980184e926f0de3c9ceab15fc059d617fa0eafe7a0c62126a4657faf596a1b211eec347',  "beaverbuild")
data =data.replace('\\x8000008a03ebae7d8ab2f66659bd719a698b2e74097d1e423df85e0d58571140527c15052a36c19878018aaebe8a6fea',  "blocknative")
data =data.replace('\\x9000009807ed12c1f08bf4e81c6da3ba8e3fc3d953898ce0102433094e5f22f21102ec057841fcb81978ed1ea0fa8246',  "blocknative")
data =data.replace('\\xa66f3abc04df65c16eb32151f2a92cb7921efdba4c25ab61b969a2af24b61508783ceb48175ef252ec9f82c6cdf8d8fd',  "blocknative")
data =data.replace('\\xa00000a975dffbd1ef61953ac6c90b52b70eb0188eb9d030774346c9248f81e875f7e8bc56c4bbbda297a9543cfa051d',  "blocknative")
data =data.replace('\\x8e39849ceabc8710de49b2ca7053813de18b1c12d9ee22149dac4b90b634dd7e6d1e7d3c2b4df806ce32c6228eb70a8b',  "Eden")
data =data.replace('\\xa5eec32c40cc3737d643c24982c7f097354150aac1612d4089e2e8af44dbeefaec08a11c76bd57e7d58697ad8b2bbef5',  "Eden")
data =data.replace('\\x91970c2db7c12510acb2e9c45844f7de602f83a7f31064f7ca04a807b607d7aebfc0abda73c036a92e5c3e56ebca04b7',  "Eden")
data =data.replace('\\xa412007971217a42ca2ced9a90e7ca0ddfc922a1482ee6adf812c4a307e5fb7d6e668a7c86e53663ddd53c689aa3d350',  "Eden")
data =data.replace('\\x82ba7cadcdfc1b156ba2c48c1c627428ba917858e62c3a97d8f919510da23d0f11cf5db53cb92a5faf5de7d31bf38632',  "Builder 2")
data =data.replace('\\xafc9274fe595e8cff421ab9e73b031f0dff707ea1852e2233ff070ef18e3876e25c44a9831c4b5f802653d4678ccc31f',  "Builder 3")
data =data.replace('\\xa1f10d66aa4b73c5d9a6cc38a098b2c6ce031a6750ea2da01918ba3ac57c2ce1e39a0da622bd8ccd7c9930861f949fa2',  "Builder 4")
data =data.replace('\\x8bcd1148e83d0a844d2d42f90df0837dbe407055367b3bfcf04227e47ea65a0164fc13a66584aae286f8f7322dc69501',  "Builder 5")
data =data.replace('\\xa25f5d5bd4f1956971bbd6e5a19e59c9b1422ca253587bbbb644645bd2067cc08fb854a231061f8c91f110254664e943',  "Builder 6")
data =data.replace('\\xa0d0dbdf7b5eda08c921dee5da7c78c34c9685db3e39e81eb91da94af29eaa50f1468813c86503bf41b4b51bf772800e',  "Manta-builder")
data =data.replace('\\xb1b734b8dd42b4744dc98ea330c3d9da64b7afc050afed96875593c73937d530a773e35ddc4b480f9d2e1d5ba452a469',  "Manta-builder")
data =data.replace('\\xb5a688d26d7858b38c44f44568d68fb94f112fc834cd225d32dc52f0277c2007babc861f6f157a6fc6c1dc25bf409046',  "Manta-builder")
data =data.replace('\\x8eb772d96a747ba63af7acdf92dc775a859f76a77e4c6ed124dca6360e74e4e798a75a925eb8fd0dde866317fff18ad0',  "eth-builder")
data =data.replace('\\x8ea1393f49d894ae22ec86e38d9aeb64b8336dac947e69cb8468acf510d010ce0b51b21ac3e1244bdb91c52e020ea525',  "eth-builder")
data =data.replace('\\x945fc51bf63613257792926c9155d7ae32db73155dc13bdfe61cd476f1fd2297b66601e8721b723cef11e4e6682e9d87',  "rsync-builder")
data =data.replace('\\x83d3495a2951065cf19c4d282afca0a635a39f6504bd76282ed0138fe28680ec60fa3fd149e6d27a94a7d90e7b1fb640',  "rsync-builder")
data =data.replace('\\x978a35c39c41aadbe35ea29712bccffb117cc6ebcad4d86ea463d712af1dc80131d0c650dc29ba29ef27c881f43bd587',  "rsync-builder")        

In [ ]:
blockPerBuilder=data[data['PBSBlock']==1]['builderPubkey'].value_counts()
top11 =blockPerBuilder[:11].index.tolist()


blocksPerBuilder=data.copy()
blocksPerBuilder.loc[(~blocksPerBuilder['builderPubkey'].isin(top11)) &blocksPerBuilder['PBSBlock']==1 ,'builderPubkey']= 'rest'
blocksPerBuilder.loc[blocksPerBuilder['PBSBlock']==0 ,'builderPubkey']= 'no PBS'

blocksPerBuilder=blocksPerBuilder.groupby(['day', 'builderPubkey']).size().unstack(fill_value=0)
blocksPerBuilder['sum']=blocksPerBuilder.sum(axis=1)
blocksPerBuilder=blocksPerBuilder.drop(['no PBS'],axis=1)
blocksPerBuilder.iloc[:,:-1]=blocksPerBuilder.iloc[:,:-1].div(blocksPerBuilder.iloc[:,-1], axis=0)

s = blocksPerBuilder.sum()
blocksPerBuilder=blocksPerBuilder[s.sort_values(ascending=False).index]
blocksPerBuilder['rest'] = blocksPerBuilder.pop('rest')

In [ ]:
ax = blocksPerBuilder.drop('sum', axis=1).plot.area(linewidth=0,color={'Flashbots': Color.Flashbots.value, 
                                    'builder0x69': Color.builder0x69.value, 
                                    'bloXroute (M)': Color.bloXrouteM.value, 
                                    'beaverbuild': Color.beaverbuild.value, 
                                    'Builder 1': Color.MEVBuilder1.value, 
                                    'blocknative': Color.MEVBuilder2.value, 
                                    'bloXroute (R)': Color.bloXrouteR.value, 
                                    'eth-builder': Color.ethbuilder.value, 
                                    'Eden': Color.Eden.value, 
                                    'Manta-builder': Color.LIDO.value, 
                                    'bloXroute (E)': Color.bloXrouteE.value, 
                                    'rsync-builder': Color.bloXrouteE.value, 
                                    'rest': Color.REST.value})
locator = mdates.AutoDateLocator()
formatter = mdates.ConciseDateFormatter(locator)
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(formatter)   
plt.xlabel('')
plt.ylabel('share of blocks')
plt.ylim(0,1)

handles, labels = ax.get_legend_handles_labels()
lgd = ax.legend(handles,labels,facecolor ="white", framealpha=1, bbox_to_anchor=(0., 1.02, 1., .102), loc=3,  ncol=3, mode="expand", borderaxespad=0.)
ax.set_xlim([pd.to_datetime('2022-09-15'), pd.to_datetime('2023-03-31')])
plt.show()

### HHI

In [ ]:
def hhi(series):
    cnt=series
    return np.square(cnt/cnt.sum()).sum()    

blockPerBuilderHHI=data[data['PBSBlock']==1].groupby(['day', 'builderPubkey']).size()
builderHHI=blockPerBuilderHHI.to_frame().groupby(['day']).agg({0:hhi}) 
builderHHI.index=pd.to_datetime(builderHHI.index)


relayDictDataHHI =pd.DataFrame.from_dict(relaysBlockDict, orient='index').T.unstack().dropna().reset_index(level=1,drop=True)
relayDictDataHHI =relayDictDataHHI.to_frame()
relayDictDataHHI = relayDictDataHHI.reset_index()
relayDictDataHHI.columns =['block', 'Relay']
relayDictDataHHI =relayDictDataHHI.groupby(['block',  'Relay']).size().unstack(fill_value=0)
relayDictDataHHI =relayDictDataHHI.div(relayDictData.sum(axis=1), axis=0)
relayDictDataHHI=relayDictDataHHI.stack().to_frame()
relayDictDataHHI=relayDictDataHHI.reset_index(level='Relay')
relayDictDataHHI=relayDictDataHHI.merge(data[['time','block']],left_on='block',right_on='block',how='inner')
relayDictDataHHI=relayDictDataHHI.set_index('time')
relayDictDataHHI.index=pd.to_datetime(relayDictDataHHI.index)
relayDictDataHHI['day'] = relayDictDataHHI.index.date
relayDictDataHHI=relayDictDataHHI[relayDictDataHHI.iloc[:,2]!=0]
relayDictDataHHI=relayDictDataHHI.groupby(['day', 'Relay']).size()
relayDictDataHHI=relayDictDataHHI.droplevel(1, axis=0) 

relayHHI=relayDictDataHHI.to_frame().groupby(['day']).agg({0:hhi}) 
relayHHI.index=pd.to_datetime(relayHHI.index)


In [ ]:
ax=plt.gca()
plt.plot(relayHHI,color=Color.Flashbots.value,label='relays')

plt.plot(builderHHI,color=Color.bloXrouteM.value,label='builders')

locator = mdates.AutoDateLocator()
formatter = mdates.ConciseDateFormatter(locator)
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(formatter)   
plt.xlabel('')
plt.ylabel('HHI')

ax.set_xlim([pd.to_datetime('2022-09-15'), pd.to_datetime('2023-03-31')])

handles, labels = ax.get_legend_handles_labels()
lgd = ax.legend(handles,labels,facecolor ="white", framealpha=1, bbox_to_anchor=(0., 1.02, 1., .102), loc=3,  ncol=2, mode="expand", borderaxespad=0.)
plt.show()

### Builders per Relay

In [ ]:
builderPerRelay = pd.DataFrame()
builderPerRelay.index =pd.date_range(

    start=pd.to_datetime("15/09/2022",dayfirst=True),

    end=pd.to_datetime("31/03/2023",dayfirst=True),

)

relayDataBuilder =pd.DataFrame.from_dict(relaysBlockDict, orient='index').T.unstack().dropna().reset_index(level=1,drop=True)
relayDataBuilder =relayDataBuilder.to_frame()
relayDataBuilder = relayDataBuilder.reset_index()
relayDataBuilder.columns =['block', 'Relay']
relayDataBuilder=relayDataBuilder.merge(data[['block','builderPubkey','day']],left_on='block',right_on='block')


In [ ]:
relays=relayDataBuilder['Relay'].unique().tolist()
for relay in relays: 
    tempdf = relayDataBuilder[relayDataBuilder['Relay']==relay]
    tempdf=tempdf[['builderPubkey','day']].groupby('day').nunique()
    builderPerRelay[relay]=tempdf['builderPubkey']
builderPerRelay =builderPerRelay.rename(columns={
                  "bloXroute (MaxProfit)": "bloXroute (M)"
                  ,"bloXroute (Regulated)": "bloXroute (R)"
                  ,"bloXroute (Ethical)": "bloXroute (E)"
                  })

In [ ]:
ax = builderPerRelay[['Flashbots','bloXroute (M)','Blocknative','bloXroute (R)','Eden','bloXroute (E)','UltraSound','GnosisDAO']].plot(color={'Flashbots': Color.Flashbots.value, 
                                    'bloXroute (M)': Color.bloXrouteM.value, 
                                    'Blocknative': Color.beaverbuild.value, 
                                    'bloXroute (R)': Color.bloXrouteR.value, 
                                    'Eden': Color.Eden.value, 
                                    'bloXroute (E)': Color.bloXrouteE.value, 
                                    'UltraSound': Color.MEVBuilder2.value, 
                                    'GnosisDAO': Color.MEVBuilder1.value})
locator = mdates.AutoDateLocator()
formatter = mdates.ConciseDateFormatter(locator)
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(formatter)   
ax.xaxis.set_minor_formatter('')
ax.tick_params(which='minor', length=0, color='r')
plt.xlabel('')
plt.ylabel('number of builders per relay')
handles, labels = ax.get_legend_handles_labels()
lgd = ax.legend(handles,labels,facecolor ="white", framealpha=1, bbox_to_anchor=(0., 1.02, 1., .102), loc=3,  ncol=3, mode="expand", borderaxespad=0.)
ax.set_xlim([pd.to_datetime('2022-09-15'), pd.to_datetime('2023-03-31')])
plt.show()

## PBS Impact on Block Composition

### Block Value

In [ ]:
ax=sns.scatterplot(data=PBSblocks, x="time", y="blockValue",s=1,label ='PBS',linewidth=0, alpha = 0.7,color =Color.Flashbots.value)
sns.scatterplot(data=noPBSblocks, x="time", y="blockValue",s=1,label ='non-PBS',linewidth=0, alpha = 0.7,color =Color.bloXrouteM.value)
locator = mdates.AutoDateLocator()
formatter = mdates.ConciseDateFormatter(locator)
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(formatter)   
ax.xaxis.set_minor_formatter('')
ax.grid(False)

plt.xlabel('')
plt.yscale('log')
plt.ylabel('block value [ETH]')
ax.set_xlim([pd.to_datetime('2022-09-15'), pd.to_datetime('2023-03-31')])

handles, labels = ax.get_legend_handles_labels()
lgd = ax.legend(handles,labels,facecolor ="white", framealpha=1, bbox_to_anchor=(0., 1.02, 1., .102), loc=3,  ncol=3, mode="expand", borderaxespad=0.,markerscale=4)
plt.show()

### Distribution of Value

Median Proposer Profit

In [ ]:
ax =sns.lineplot(x='day', y='proposerProfit', err_style="band", 
             errorbar=None, estimator="median", data=PBSblocks,label='PBS',color =Color.Flashbots.value)
bounds = PBSblocks.groupby('day')['proposerProfit'].quantile((0.25,0.75)).unstack()
ax.fill_between(x=bounds.index,y1=bounds.iloc[:,0],y2=bounds.iloc[:,1],alpha=0.2)
ax =sns.lineplot(x='day', y='proposerProfit', err_style="band", 
             errorbar=None, estimator="median", data=noPBSblocks,label='non-PBS',color =Color.bloXrouteM.value)
bounds = noPBSblocks.groupby('day')['proposerProfit'].quantile((0.25,0.75)).unstack()
ax.fill_between(x=bounds.index,y1=bounds.iloc[:,0],y2=bounds.iloc[:,1],alpha=0.2)

locator = mdates.AutoDateLocator()
formatter = mdates.ConciseDateFormatter(locator)
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(formatter)   
ax.xaxis.set_minor_formatter('')
ax.grid(False)
plt.xlabel('')
plt.ylabel('median proposer profit [ETH]')
ax.set_xlim([pd.to_datetime('2022-09-15'), pd.to_datetime('2023-03-31')])
plt.axvline(pd.to_datetime('2022-11-09'),color =Color.beaverbuild.value, linestyle='dashed',label='FTX bankrupcy')
plt.axvline(pd.to_datetime('2023-03-11'),color =Color.MEVBuilder2.value, linestyle='dashed',label='USDC depeg')

plt.ylim(0,0.3)
handles, labels = ax.get_legend_handles_labels()
lgd = ax.legend(handles,labels,facecolor ="white", framealpha=1, bbox_to_anchor=(0., 1.02, 1., .102), loc=3,  ncol=2, mode="expand", borderaxespad=0.)
plt.show()

Builder Profit

In [ ]:
dataProfit =data[['builderProfit','builderPubkey','PBSBlock','proposerProfit']]
dataProfit.loc[(~dataProfit['builderPubkey'].isin(top11)) &dataProfit['PBSBlock']==1 ,'builderPubkey']= 'rest'
dataProfit.loc[dataProfit['PBSBlock']==0 ,'builderPubkey']= 'no PBS'
dataProfit=dataProfit[dataProfit['builderPubkey']!='no PBS']
dataProfit =  dataProfit[dataProfit['builderPubkey'].isin(top11)]
dataProfit = dataProfit[dataProfit['builderPubkey']!='rest']

In [ ]:
plt.axhline(y=0, linewidth=1, color='gray',zorder=1)
my_pal = {'Flashbots': Color.Flashbots.value, 
                                    'builder0x69': Color.builder0x69.value, 
                                    'bloXroute (M)': Color.bloXrouteM.value, 
                                    'beaverbuild': Color.beaverbuild.value, 
                                    'Builder 1': Color.MEVBuilder1.value, 
                                    'blocknative': Color.MEVBuilder2.value, 
                                    'bloXroute (R)': Color.bloXrouteR.value, 
                                    'eth-builder': Color.ethbuilder.value, 
                                    'Eden': Color.Eden.value, 
                                    'Manta-builder': Color.LIDO.value, 
                                    'bloXroute (E)': Color.bloXrouteE.value, 
                                    'rsync-builder': Color.bloXrouteE.value, 
                                    'rest': Color.REST.value}
ax = sns.boxplot(x="builderPubkey", y="builderProfit", data=dataProfit,order=top11, showfliers=False, palette=my_pal, linewidth=1,showmeans=True, meanprops={"marker":"o",'color': 'k',"markerfacecolor":"black", "markeredgecolor":"black",'markersize':4})#,showmeans=True,meanprops={"marker":"o","markersize":"4"})

ax.set_xticklabels(ax.get_xticklabels(),rotation=30,horizontalalignment='right')
plt.xlabel('')
plt.ylabel('builder profit [ETH]')
plt.grid(False)
plt.show()

Proposer Profit by Builder

In [ ]:
plt.axhline(y=0, linewidth=1, color='gray',zorder=1)
my_pal = {'Flashbots': Color.Flashbots.value, 
                                    'builder0x69': Color.builder0x69.value, 
                                    'bloXroute (M)': Color.bloXrouteM.value, 
                                    'beaverbuild': Color.beaverbuild.value, 
                                    'Builder 1': Color.MEVBuilder1.value, 
                                    'blocknative': Color.MEVBuilder2.value, 
                                    'bloXroute (R)': Color.bloXrouteR.value, 
                                    'eth-builder': Color.ethbuilder.value, 
                                    'Eden': Color.Eden.value, 
                                    'Manta-builder': Color.LIDO.value, 
                                    'bloXroute (E)': Color.bloXrouteE.value, 
                                    'rsync-builder': Color.bloXrouteE.value, 
                                    'rest': Color.REST.value}

ax = sns.boxplot(x="builderPubkey", y="proposerProfit", data=dataProfit,order=top11, showfliers=False, palette=my_pal, linewidth=1,showmeans=True, meanprops={"marker":"o",'color': 'k',"markerfacecolor":"black", "markeredgecolor":"black",'markersize':4})#,showmeans=True,meanprops={"marker":"o","markersize":"4"})
ax.set_xticklabels(ax.get_xticklabels(),rotation=30,horizontalalignment='right')
plt.xlabel('')
plt.ylabel('proposer profit [ETH]')
plt.grid(False)
plt.show()

In [ ]:
blockValue =data[data['PBSBlock'] ==1].groupby('day').aggregate({'builderProfit':sum,'proposerProfit':sum})
blockValue['builderprofitpercent'] =blockValue['builderProfit']/(blockValue['builderProfit']+blockValue['proposerProfit'])
blockValue['proposerprofitpercent'] =blockValue['proposerProfit']/(blockValue['builderProfit']+blockValue['proposerProfit'])

conditions = [
    blockValue['builderprofitpercent'] >0
]
strategies = [blockValue['builderprofitpercent']]
blockValue['a'] = np.select(conditions, strategies, default=0)

conditions = [
    blockValue['builderprofitpercent'] <0
]
strategies = [blockValue['builderprofitpercent']]
blockValue['b'] = np.select(conditions, strategies, default=0)

plt.stackplot(blockValue.index.tolist(),  blockValue.a.tolist(),blockValue.proposerprofitpercent.tolist(),
              labels=[  "builder profit","proposer profit" ], 
              colors =[Color.Flashbots.value, Color.bloXrouteM.value])

plt.stackplot(blockValue.index.tolist(), blockValue.b.tolist(),colors =[Color.Flashbots.value])
ax =plt.gca()
locator = mdates.AutoDateLocator()
formatter = mdates.ConciseDateFormatter(locator)
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(formatter)     
plt.xlabel('')
plt.ylabel('profit share')

handles, labels = ax.get_legend_handles_labels()
lgd = ax.legend(handles,labels,facecolor ="white", framealpha=1, bbox_to_anchor=(0., 1.02, 1., .102), loc=3,  ncol=6, mode="expand", borderaxespad=0.)
ax.set_xlim([pd.to_datetime('2022-09-15'), pd.to_datetime('2023-03-31')])

plt.show()

### Block Contents

Gas Usage

In [ ]:
ax =sns.lineplot(x='day', y='gasUsed', 
             errorbar=None, estimator="mean", data=PBSblocks,label='PBS',color =Color.Flashbots.value)
ax =sns.lineplot(x='day', y='gasUsed', 
             errorbar=None, estimator="mean", data=noPBSblocks,label='non-PBS',color =Color.bloXrouteM.value)
bounds = PBSblocks.groupby('day')['gasUsed'].agg([np.mean, np.std])

bounds['lower']=bounds['mean']-1*bounds['std']
bounds['upper']=bounds['mean']+1*bounds['std']
bounds =bounds[['lower','upper']]
ax.fill_between(x=bounds.index,y1=bounds.iloc[:,0],y2=bounds.iloc[:,1],alpha=0.2)
bounds = noPBSblocks.groupby('day')['gasUsed'].agg([np.mean, np.std])
bounds['lower']=bounds['mean']-1*bounds['std']
bounds['upper']=bounds['mean']+1*bounds['std']
bounds =bounds[['lower','upper']]
ax.fill_between(x=bounds.index,y1=bounds.iloc[:,0],y2=bounds.iloc[:,1],alpha=0.2)
locator = mdates.AutoDateLocator()
formatter = mdates.ConciseDateFormatter(locator)
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(formatter)   
ax.xaxis.set_minor_formatter('')
ax.grid(False)

formatter = mticker.ScalarFormatter(useMathText=True)
formatter.set_powerlimits((-3,3))
ax.yaxis.set_major_formatter(formatter)   
lo = Labeloffset(ax, label=r"block size [gas]", axis="y")
plt.xlabel('')
plt.axhline(y=15000000, color='gray', linestyle='--',label='target block size')
ax.set_xlim([pd.to_datetime('2022-09-15'), pd.to_datetime('2023-03-31')])
handles, labels = ax.get_legend_handles_labels()
lgd = ax.legend(handles,labels,facecolor ="white", framealpha=1, bbox_to_anchor=(0., 1.02, 1., .102), loc=3,  ncol=3, mode="expand", borderaxespad=0.)
plt.show()

Private Transactions

In [ ]:
data[['publicTransactions','transactions']]

conditions = [
     ( data['payment']==1)
]
strategies = [data['transactions']-data['publicTransactions']]
data['privateTransactions'] = np.select(conditions, strategies, default=data['transactions']-data['publicTransactions']-1)
PBSblocks =data[data['PBSBlock']==1]
noPBSblocks =data[data['PBSBlock']==0]
PBSBlocksPrivate=PBSblocks[['privateTransactions','transactions','day']].groupby('day').sum()
noPBSblocksPrivate=noPBSblocks[['privateTransactions','transactions','day']].groupby('day').sum()

PBSBlocksPrivate['percentPrivate'] =PBSBlocksPrivate['privateTransactions']/PBSBlocksPrivate['transactions']
noPBSblocksPrivate['percentPrivate'] =noPBSblocksPrivate['privateTransactions']/noPBSblocksPrivate['transactions']

In [ ]:
ax =sns.lineplot(x='day', y='percentPrivate', 
             errorbar=None, estimator="mean", data=PBSBlocksPrivate,label='PBS',color =Color.Flashbots.value)
ax =sns.lineplot(x='day', y='percentPrivate', 
             errorbar=None, estimator="mean", data=noPBSblocksPrivate,label='non-PBS',color =Color.bloXrouteM.value)


locator = mdates.AutoDateLocator()
formatter = mdates.ConciseDateFormatter(locator)
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(formatter)   
ax.xaxis.set_minor_formatter('')
ax.grid(False)

formatter = mticker.ScalarFormatter(useMathText=True)
formatter.set_powerlimits((-3,3))
ax.yaxis.set_major_formatter(formatter)   
lo = Labeloffset(ax, label=r"daily share of private txs", axis="y")
plt.xlabel('')
ax.set_xlim([pd.to_datetime('2022-09-15'), pd.to_datetime('2023-03-31')])
handles, labels = ax.get_legend_handles_labels()
lgd = ax.legend(handles,labels,facecolor ="white", framealpha=1, bbox_to_anchor=(0., 1.02, 1., .102), loc=3,  ncol=3, mode="expand", borderaxespad=0.)
plt.show()

In [ ]:
privateNonPBS = privateNonPBS.loc['2022-12-01':'2022-12-31']
print("Number of private txs in December in non-PBS blocks: "+str(len(privateNonPBS)))
binanceTxs =privateNonPBS[(privateNonPBS['sender']=='\\x4d9ff50ef4da947364bb9650892b2554e7be5e2b')&(privateNonPBS['recipient']=='\\x0b95993a39a363d99280ac950f5e4536ab5c5566')]
print("Number of private txs with sender/receiver pair: "+str(len(binanceTxs)))
binanceTxs=binanceTxs.merge(data,left_on='block',right_on='block',how ='inner')
print("Number of private txs in ANKR block: "+str(len(binanceTxs[binanceTxs['proposerFeeRecipient']=='\\x6a0db4cef1ce2a5f81c8e6322862439f71aca29d'])))
print("Number of unique ANKR block: "+str(binanceTxs[binanceTxs['proposerFeeRecipient']=='\\x6a0db4cef1ce2a5f81c8e6322862439f71aca29d'].block.nunique()))

### MEV Value

MEV Transactions

In [ ]:
ax=sns.lineplot(x='day', y='mevTxs', err_style="band",errorbar=None, estimator="mean", data=PBSblocks,label='PBS',color =Color.Flashbots.value)

ax=sns.lineplot(x='day', y='mevTxs', err_style="band",errorbar=None, estimator="mean", data=noPBSblocks,label='non-PBS',color =Color.bloXrouteM.value)
locator = mdates.AutoDateLocator()
formatter = mdates.ConciseDateFormatter(locator)
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(formatter)   
ax.xaxis.set_minor_formatter('')
plt.xlabel('')
plt.ylabel('mean # MEV per block')

handles, labels = ax.get_legend_handles_labels()
lgd = ax.legend(handles,labels,facecolor ="white", framealpha=1, bbox_to_anchor=(0., 1.02, 1., .102), loc=3,  ncol=6, mode="expand", borderaxespad=0.)
ax.set_xlim([pd.to_datetime('2022-09-15'), pd.to_datetime('2023-03-31')])
plt.show()

Sandwich Attacks

In [ ]:
ax=sns.lineplot(x='day', y='sandwichTxs', err_style="band",errorbar=None, estimator="mean", data=PBSblocks,label='PBS',color =Color.Flashbots.value)

ax=sns.lineplot(x='day', y='sandwichTxs', err_style="band",errorbar=None, estimator="mean", data=noPBSblocks,label='non-PBS',color =Color.bloXrouteM.value)
locator = mdates.AutoDateLocator()
formatter = mdates.ConciseDateFormatter(locator)
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(formatter)   
ax.xaxis.set_minor_formatter('')
plt.xlabel('')
plt.ylabel('mean # sandwich attacks per block')

handles, labels = ax.get_legend_handles_labels()
lgd = ax.legend(handles,labels,facecolor ="white", framealpha=1, bbox_to_anchor=(0., 1.02, 1., .102), loc=3,  ncol=6, mode="expand", borderaxespad=0.)
ax.set_xlim([pd.to_datetime('2022-09-15'), pd.to_datetime('2023-03-31')])
plt.show()

In [ ]:
print("Average number of sandwich attacks in PBS blocks: ",PBSblocks['sandwichTxs'].mean())
print("Average number of sandwich attacks in non-PBS blocks: ",noPBSblocks['sandwichTxs'].mean())
print("Total number of sandwich attacks: ",PBSblocks['sandwichTxs'].sum()+noPBSblocks['sandwichTxs'].sum())
print("Sandwich attacks in bloXroute (Ethical) blocks: ",relayDictData[relayDictData['bloXroute (Ethical)']>0].merge(PBSblocks,left_index=True,right_on='block',how='inner')['sandwichTxs'].sum())

Liquidations

In [ ]:
ax=sns.lineplot(x='day', y='liquidationTxs', err_style="band",errorbar=None, estimator="mean", data=PBSblocks,label='PBS',color =Color.Flashbots.value)

ax=sns.lineplot(x='day', y='liquidationTxs', err_style="band",errorbar=None, estimator="mean", data=noPBSblocks,label='non-PBS',color =Color.bloXrouteM.value)

locator = mdates.AutoDateLocator()
formatter = mdates.ConciseDateFormatter(locator)
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(formatter)   
ax.xaxis.set_minor_formatter('')
plt.xlabel('')
plt.ylabel('mean # liquidations per block')

handles, labels = ax.get_legend_handles_labels()
lgd = ax.legend(handles,labels,facecolor ="white", framealpha=1, bbox_to_anchor=(0., 1.02, 1., .102), loc=3,  ncol=6, mode="expand", borderaxespad=0.)
ax.set_xlim([pd.to_datetime('2022-09-15'), pd.to_datetime('2023-03-31')])
plt.show()

In [ ]:
print("Average number of liquidations in PBS blocks: ",PBSblocks['liquidationTxs'].mean())
print("Average number of liquidations in non-PBS blocks: ",noPBSblocks['liquidationTxs'].mean())
print("Total number of liquidatations: ",PBSblocks['liquidationTxs'].sum()+noPBSblocks['liquidationTxs'].sum())


Cyclic Arbitrage

In [ ]:
ax=sns.lineplot(x='day', y='arbitrageTxs', err_style="band",errorbar=None, estimator="mean", data=PBSblocks,label='PBS',color =Color.Flashbots.value)
ax=sns.lineplot(x='day', y='arbitrageTxs', err_style="band",errorbar=None, estimator="mean", data=noPBSblocks,label='non-PBS',color =Color.bloXrouteM.value)

locator = mdates.AutoDateLocator()
formatter = mdates.ConciseDateFormatter(locator)
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(formatter)   
ax.xaxis.set_minor_formatter('')
plt.xlabel('')
plt.ylabel('mean # cylic arbitrage per block')

handles, labels = ax.get_legend_handles_labels()
lgd = ax.legend(handles,labels,facecolor ="white", framealpha=1, bbox_to_anchor=(0., 1.02, 1., .102), loc=3,  ncol=6, mode="expand", borderaxespad=0.)
ax.set_xlim([pd.to_datetime('2022-09-15'), pd.to_datetime('2023-03-31')])
plt.show()

In [ ]:
print("Average number of cylic arbitrage transactions in PBS blocks: ",PBSblocks['arbitrageTxs'].mean())
print("Average number of cylic arbitrage transactions in non-PBS blocks: ",noPBSblocks['arbitrageTxs'].mean())
print("Total number of cylic arbitrage transactions: ",PBSblocks['arbitrageTxs'].sum()+noPBSblocks['arbitrageTxs'].sum())


Proportion of MEV value

In [ ]:
PBSblocks['mevPop'] =PBSblocks['mevFee']/PBSblocks['blockValue']

noPBSblocks['mevPop'] =noPBSblocks['mevFee']/noPBSblocks['blockValue']


ax =sns.lineplot(x='day', y='mevPop', err_style="band", 
             errorbar=None, estimator="mean", data=PBSblocks,label='PBS',color =Color.Flashbots.value)
bounds = PBSblocks.groupby('day')['mevPop'].quantile((0.25,0.75)).unstack()
ax.fill_between(x=bounds.index,y1=bounds.iloc[:,0],y2=bounds.iloc[:,1],alpha=0.2)
ax =sns.lineplot(x='day', y='mevPop', err_style="band", 
             errorbar=None, estimator="mean", data=noPBSblocks,label='non-PBS',color =Color.bloXrouteM.value)
bounds = noPBSblocks.groupby('day')['mevPop'].quantile((0.25,0.75)).unstack()
ax.fill_between(x=bounds.index,y1=bounds.iloc[:,0],y2=bounds.iloc[:,1],alpha=0.2)

locator = mdates.AutoDateLocator()
formatter = mdates.ConciseDateFormatter(locator)
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(formatter)   
ax.xaxis.set_minor_formatter('')
ax.grid(False)
plt.xlabel('')
plt.ylabel('mean MEV share of block value')
ax.set_xlim([pd.to_datetime('2022-09-15'), pd.to_datetime('2023-03-31')])

handles, labels = ax.get_legend_handles_labels()
lgd = ax.legend(handles,labels,facecolor ="white", framealpha=1, bbox_to_anchor=(0., 1.02, 1., .102), loc=3,  ncol=2, mode="expand", borderaxespad=0.)

plt.show()

In [ ]:
print("Proportion of value from MEV in PBS blocks: ",PBSblocks['mevPop'].mean())
print("Proportion of value from MEV in non-PBS blocks: ",noPBSblocks['mevPop'].mean())

## Censorship Resistance

### Blocks by Relay OFAC Compliance

In [ ]:
dataRelayOFAC = dataRelayTop.copy()
dataRelayOFAC['non OFAC-compliant'] = dataRelayOFAC['bloXroute (M)']+ dataRelayOFAC['bloXroute (E)']+ dataRelayOFAC['UltraSound']+ dataRelayOFAC['GnosisDAO']+ dataRelayOFAC['rest']
dataRelayOFAC['OFAC-compliant'] = dataRelayOFAC['Flashbots']+dataRelayOFAC['Eden']+dataRelayOFAC['Blocknative']+dataRelayOFAC['bloXroute (R)']
dataRelayOFAC['sum']=dataRelayOFAC['OFAC-compliant']+dataRelayOFAC['non OFAC-compliant']
dataRelayOFAC['non OFAC-compliant'] = dataRelayOFAC['non OFAC-compliant']/dataRelayOFAC['sum']
dataRelayOFAC['OFAC-compliant'] = dataRelayOFAC['OFAC-compliant']/dataRelayOFAC['sum']
dataRelayOFAC = dataRelayOFAC[['non OFAC-compliant','OFAC-compliant']]

In [ ]:
ax = dataRelayOFAC.plot.area(linewidth=0,color={'non OFAC-compliant': Color.Flashbots.value, 
                                    'OFAC-compliant': Color.bloXrouteM.value})
locator = mdates.AutoDateLocator()
formatter = mdates.ConciseDateFormatter(locator)
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(formatter)   
ax.xaxis.set_minor_formatter('')
ax.tick_params(which='minor', length=0, color='r')
plt.xlabel('')
plt.ylabel('share of PBS blocks')
plt.ylim(0,1)
handles, labels = ax.get_legend_handles_labels()
lgd = ax.legend(handles,labels,facecolor ="white", framealpha=1, bbox_to_anchor=(0., 1.02, 1., .102), loc=3,  ncol=3, mode="expand", borderaxespad=0.)
ax.set_xlim([pd.to_datetime('2022-09-15'), pd.to_datetime('2023-03-31')])
plt.show()

In [ ]:
conditions = [
    noPBSblocks['PBSBlock'] ==1
]
strategies = [0]
noPBSblocks['noPBSBlock'] = np.select(conditions, strategies, default=1)

PBSOfac=PBSblocks.groupby('day').aggregate({'ofac':sum,'PBSBlock':sum})
PBSOfac['percentOfac']= PBSOfac['ofac']/PBSOfac['PBSBlock']
noPBSOfac=noPBSblocks.groupby('day').aggregate({'ofac':sum, 'noPBSBlock':sum})
noPBSOfac['percentOfac']= noPBSOfac['ofac']/noPBSOfac['noPBSBlock']


In [ ]:
print("Proportion of non OFAC compliant PBS blocks: " +str(PBSOfac['percentOfac'].mean()))
print("Proportion of non OFAC compliant non-PBS blocks: " +str(noPBSOfac['percentOfac'].mean()))

### Share of Sanctioned Blocks

In [ ]:
ax =sns.lineplot(x='day', y='percentOfac', 
             errorbar=None, estimator="mean", data=PBSOfac,label='PBS',color =Color.Flashbots.value)
ax =sns.lineplot(x='day', y='percentOfac', 
             errorbar=None, estimator="mean", data=noPBSOfac,label='non-PBS',color =Color.bloXrouteM.value)


locator = mdates.AutoDateLocator()
formatter = mdates.ConciseDateFormatter(locator)
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(formatter)   
ax.xaxis.set_minor_formatter('')
ax.grid(False)

formatter = mticker.ScalarFormatter(useMathText=True)
formatter.set_powerlimits((-3,3))
ax.yaxis.set_major_formatter(formatter)   
lo = Labeloffset(ax, label=r"share of sanctioned blocks", axis="y")
plt.xlabel('')
ax.set_xlim([pd.to_datetime('2022-09-15'), pd.to_datetime('2023-03-31')])
handles, labels = ax.get_legend_handles_labels()
lgd = ax.legend(handles,labels,facecolor ="white", framealpha=1, bbox_to_anchor=(0., 1.02, 1., .102), loc=3,  ncol=3, mode="expand", borderaxespad=0.)
plt.show()

## Relay Delivered Value and OFAC Compliance

In [ ]:
relayDictData =pd.DataFrame.from_dict(relaysBlockDict, orient='index').T.unstack().dropna().reset_index(level=1,drop=True)
relayDictData =relayDictData.to_frame()
relayDictData = relayDictData.reset_index()
relayDictData.columns =['block', 'Relay']
relayDictData =relayDictData.groupby(['block',  'Relay']).size().unstack(fill_value=0)
dataRelayCensor = relayDictData.merge(data[['block','time']],left_on='block',right_on='block',how='inner')

ofacblocks =data[data['ofac']==1]
ofacRelays =relayDictData.merge(ofacblocks['block'],left_index=True,right_on='block')
ofacRelays=ofacRelays.drop(['block'], axis =1)
ofacRelays =ofacRelays.rename(columns={
                  "bloXroute (MaxProfit)": "bloXroute (M)"
                  ,"bloXroute (Regulated)": "bloXroute (R)"
                  ,"bloXroute (Ethical)": "bloXroute (E)"
                  })
counts1 = ofacRelays.sum()
counts1=counts1.sort_values(ascending=False)


dataRelayCensor =dataRelayCensor.set_index('time')
dataRelayCensor.index = pd.to_datetime(dataRelayCensor.index)
dataRelayCensor['day'] = dataRelayCensor.index.date
dataRelayCensor =dataRelayCensor.rename(columns={
                  "bloXroute (MaxProfit)": "bloXroute (M)"
                  ,"bloXroute (Regulated)": "bloXroute (R)"
                  ,"bloXroute (Ethical)": "bloXroute (E)"
                  })
dataRelayCensor=dataRelayCensor.drop(columns=['block']).sum()
dataRelayCensor.loc['all'] = dataRelayCensor.sum()
a=counts1.rename('ofac').to_frame()#.join(counts2.rename('chainalysis'))
a.loc['all', :] = a.sum()
b=a.div(dataRelayCensor, axis=0).mul(100)
c=pd.concat([a,b],axis=1,keys=['sanctioned blocks','share of sanctioned blocks [%]']).swaplevel(0,1,axis=1).sort_index(axis=1)
c.columns = c.columns.droplevel()

In [ ]:
missingvalue =data[(data['builderPubkey']==data['builderPubkey'])&(data['blockHashPBS']==data['blockHashOnChain'])]
conditions = [
    ( missingvalue['payment']==1),
    ( missingvalue['payment']=='')&(missingvalue['builderFeeRecipient']==missingvalue['builderFeeRecipient'])
]
strategies = [missingvalue['proposerProfit'],missingvalue['proposerProfit']]
missingvalue['deliveredValue'] = np.select(conditions, strategies, default=0)
relayDictDataValue =relayDictData.merge(missingvalue[['block','valuePBS','deliveredValue']],left_index=True,right_on='block')
relayDictDataValue =relayDictDataValue.rename(columns={
                  "bloXroute (MaxProfit)": "bloXroute (M)"
                  ,"bloXroute (Regulated)": "bloXroute (R)"
                  ,"bloXroute (Ethical)": "bloXroute (E)"
                  })
onlySmaller=relayDictDataValue[relayDictDataValue['valuePBS']>1.00000000000001*relayDictDataValue['deliveredValue']]

In [ ]:
delValue =relayDictDataValue.iloc[:,:-2].mul(relayDictDataValue.iloc[:,-1], axis=0).sum().rename('delivered Value')
delValue.loc['all'] = delValue.sum()
promisedValue=relayDictDataValue.iloc[:,:-2].mul(relayDictDataValue.iloc[:,-2], axis=0).fillna(0).sum().rename('promised Value')
promisedValue.loc['all'] = promisedValue.sum()
share =(relayDictDataValue.iloc[:,:-2].mul(relayDictDataValue.iloc[:,-1], axis=0).sum()*100/relayDictDataValue.iloc[:,:-2].mul(relayDictDataValue.iloc[:,-2], axis=0).sum()).rename('share of value')
share['all'] =delValue.loc['all']*100 /promisedValue.loc['all']
shareOfBlocks =onlySmaller.iloc[:,:-2].sum()*100/relayDictDataValue.iloc[:,:-2].sum()
shareOfBlocks['all'] = len(onlySmaller)*100 / len(relayDictDataValue)
shareOfBlocks=shareOfBlocks.rename('share of block')

In [ ]:
delValue =relayDictDataValue.iloc[:,:-2].mul(relayDictDataValue.iloc[:,-1], axis=0).sum().rename('delivered value [ETH]')
delValue.loc['all'] = delValue.sum()
promisedValue=relayDictDataValue.iloc[:,:-2].mul(relayDictDataValue.iloc[:,-2], axis=0).fillna(0).sum().rename('promised value [ETH]')
promisedValue.loc['all'] = promisedValue.sum()
share =(relayDictDataValue.iloc[:,:-2].mul(relayDictDataValue.iloc[:,-1], axis=0).sum()*100/relayDictDataValue.iloc[:,:-2].mul(relayDictDataValue.iloc[:,-2], axis=0).sum()).rename('share of value [%]')
share['all'] =delValue.loc['all']*100 /promisedValue.loc['all']
shareOfBlocks =onlySmaller.iloc[:,:-2].sum()*100/relayDictDataValue.iloc[:,:-2].sum()
shareOfBlocks['all'] = len(onlySmaller)*100 / len(relayDictDataValue)
shareOfBlocks=shareOfBlocks.rename('share of blocks [%]')
table =delValue.to_frame().join(promisedValue).join(share).join(shareOfBlocks).join(c)
table.drop(['block'],inplace=True)
table